In [1]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from examples.train_model_on_k_fold_validation import load_configuration
from plotting.plotting import error_per_station_calendar_pattern

Training and Hyper-parameter tuning with Ray is not possible


In [2]:
modification = dict(model_name='STGCN',
                    args_init = None,
                    dataset_names=['subway_in','netmob_POIs_per_station'], # 'subway_out_per_station' ,'subway_out', 'netmob_POIs','netmob_POIs_per_station'
                    dataset_for_coverage=['subway_in','netmob_POIs'],
                    modification = {'epochs':10,
                                    'fold_to_evaluate' : [5],
                                    'stacked_contextual': True, # True # False
                                    'temporal_graph_transformer_encoder': True,
                                    'TGE_num_layers' : 4, #2
                                    'TGE_num_heads' :  4, #2
                                    'TGE_FC_hdim' :  64, #32
                                    'compute_node_attr_with_attn' : True
                                    #'vision_concatenation_early':True,   
                                    #'vision_concatenation_late':False,
                                    }
                    )

args,fold_to_evaluate = load_configuration(trial_id=None,load_config=False,modification=modification)


station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init=args,
                                                  modification=modification,
                                                  station=station,
                                                   training_mode_to_visualise=training_mode_to_visualise,
                                                   fold_to_evaluate= fold_to_evaluate
                                                   )

print(trainer.performance)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 


Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  776
Custom POIs associated by stations:  ['AMP: 34', 'BEL: 40', 'BRO: 42', 'COR: 51', 'CUI: 100', 'CUS: 29', 'FLA: 44', 'GOR: 59', 'BLA: 31', 'GRA: 82', 'GUI: 49', 'GIL: 52', 'HEN: 47', 'HOT: 56', 'LAE: 33', 'MAS: 42', 'MER: 39', 'LUM: 46', 'PRY: 27', 'PER: 42', 'SAN: 42', 'SAX: 53', 'VMY: 20', 'JEA: 37', 'BON: 43', 'CHA: 46', 'VAI: 47', 'VEN: 97', 'MAC: 36', 'GAR: 44', 'FOC: 50', 'REP: 61', 'GER: 21', 'DEB: 28', 'JAU: 24', 'CPA: 54', 'CRO: 52', 'PAR: 48', 'SOI: 66', 'OGA: 85']
vision_input_type POIs
vision_model_name None
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER'

Loading BokehJS ...

{'valid_loss': 0.0038426163546899533, 'valid_metrics': {'mse': 2040.3837890625, 'mae': 26.540943145751953, 'mape': 36.91934585571289}, 'test_metrics': {'mse': 1914.65576171875, 'mae': 25.767345428466797, 'mape': 31.885332107543945}, 'epoch': 9, 'training_over': True, 'fold': 0}


In [ ]:
# Architecture where Channels are Stacked:
if True:
    modif_add= {'stacked_contextual': True, # True # False
                    'temporal_graph_transformer_encoder': True, # False, # True
                    'TGE_num_layers' : 8, #2
                    'TGE_num_heads' :  2, #2
                    'TGE_FC_hdim' :  128, #32
                    'compute_node_attr_with_attn' : False # False, # True
                    }
    
# Architecture where Channels are represented in a latend space and then concatenated
if False:
    modif_add= {'stacked_contextual': False, # True # False
                'vision_concatenation_early':True,  # True, # False, 
                'vision_concatenation_late':False,  # True, # False, 
                'vision_model_name': 'VariableSelectionNetwork', #'VariableSelectionNetwork', # None
                    }

modification = dict(model_name='STGCN',
                    args_init = None,
                    dataset_names=['subway_in','subway_out'], # 'subway_out_per_station' ,'subway_out', 'netmob_POIs','netmob_POIs_per_station'
                    dataset_for_coverage=['subway_in','netmob_POIs'],
                    modification = {'epochs':100,
                                    'fold_to_evaluate' : [5],

                                    # TemporalGraphEncoder:
                                    'temporal_graph_transformer_encoder': True, # False, # True
                                    'TGE_num_layers' : 8, #2
                                    'TGE_num_heads' :  2, #2
                                    'TGE_FC_hdim' :  128, #32
                                    # ...

                                    }
                    )

modification.update(modif_add)

args,fold_to_evaluate = load_configuration(trial_id=None,load_config=False,modification=modification)


station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init=args,
                                                  modification=modification,
                                                  station=station,
                                                   training_mode_to_visualise=training_mode_to_visualise,
                                                   fold_to_evaluate= fold_to_evaluate
                                                   )

print(trainer.performance)

In [4]:
modification = dict(model_name='STGCN',
                    args_init = None,
                    dataset_names=['subway_in','subway_out'], # 'subway_out_per_station' ,'subway_out', 'netmob_POIs','netmob_POIs_per_station'
                    dataset_for_coverage=['subway_in','netmob_POIs'],
                    modification = {'epochs':100,
                                    'fold_to_evaluate' : [5],
                                    'stacked_contextual': True, # True # False
                                    #'vision_concatenation_early':True,   
                                    #'vision_concatenation_late':False,
                                    #'vision_model_name': 'VariableSelectionNetwork', #'VariableSelectionNetwork', # None
                                    'temporal_graph_transformer_encoder': False,
                                    'compute_node_attr_with_attn' : False
                                    #'vision_concatenation_early':True,   
                                    #'vision_concatenation_late':False,
                                    }
                    )

args,fold_to_evaluate = load_configuration(trial_id=None,load_config=False,modification=modification)


station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init=args,
                                                  modification=modification,
                                                  station=station,
                                                   training_mode_to_visualise=training_mode_to_visualise,
                                                   fold_to_evaluate= fold_to_evaluate
                                                   )

print(trainer.performance)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  776
T_subway_out:  torch.Size([7392, 40])
vision_input_type POIs
vision_model_name None
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER',
       'SAN', 'SAX', 'VMY', 'JEA', 'BON', 'CHA', 'VAI', 'VEN', 'MAC', 'GAR',
       'FOC', 'REP', 'GER', 'DEB', 'JAU', 'CPA', 'CRO', 'PAR', 'SOI', 'OGA'],
      dtype='object', name='COD_TRG')

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  481
Considered Spatial-Unit:

Loading BokehJS ...

{'valid_loss': 0.002476329245465867, 'valid_metrics': {'mse': 1211.655517578125, 'mae': 20.898967742919922, 'mape': 36.149932861328125}, 'test_metrics': {'mse': 1303.2550048828125, 'mae': 20.867507934570312, 'mape': 31.87664794921875}, 'epoch': 62, 'training_over': True, 'fold': 0}


In [5]:
import geopandas as gpd
from constants.paths import FOLDER_PATH
gdf_POI_2_tile_ids = gpd.read_file(f"{FOLDER_PATH}/POIs/gdf_POI_2_tile_ids.geojson")

gdf_plot = gdf_POI_2_tile_ids[(gdf_POI_2_tile_ids.tag == 'iris')&(gdf_POI_2_tile_ids['type'] == 'POI')].drop(columns = ['tile_ids'])
display(gdf_plot) 
gdf_POI_2_tile_ids[(gdf_POI_2_tile_ids.tag == 'iris')&(gdf_POI_2_tile_ids['type'] == 'POI')].drop(columns = ['tile_ids']).explore()

In [2]:
# Init
trial_id ='subway_in_STGCN_MSELoss_2025_01_06_08_00_94523'
args,_ = load_configuration(trial_id,load_config=True)


station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']
modification ={'keep_best_weights':True,
                'epochs':1,
                'validation_split_method' : 'forward_chaining_cv',
                'min_fold_size_proportion': 0.75,
                'train_prop':0.6,
                'valid_prop':0.2,
                'test_prop':0.2,
                'freq':'15min',
                'stacked_contextual': False,
                'data_augmentation': False, #True,  #False
                }

# Training and visu: 
fold_to_evaluate = [args.K_fold-1]
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init=args,
                                                  modification=modification,
                                                  fold_to_evaluate= fold_to_evaluate,
                                                  station=station,
                                                   training_mode_to_visualise=training_mode_to_visualise,
                                                   type_POIs = ['iris','iris','iris','iris','iris'],  #'stadium'
                                                   spatial_units = ['Charmettes', 'Charles Hernu', 'Les Brotteaux', 'Bellecombes-Thiers', 'Tonkin Sud'],  #'Matmut Stadium Gerland',
                                                   apps = ['Instagram','Google_Maps','Twitter'],
                                                   POI_or_stations = ['POI','POI','POI','POI','POI',], #'POI',
                                                   transfer_modes = ['DL','UL'],
                                                   expanded= '',
                                                   
                                                   )


>>>> Load best CONFIG
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  776
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Considered Spatial-Unit:  None
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER',
       'SAN', 'SAX', 'VMY', 'JEA', 'BON', 'CHA', 'VAI', 'VEN', 'MAC', 'GAR',
       'FOC', 'REP', 'GER', 'DEB', 'JAU', 'CPA', 'CRO', 'PAR', 'SOI', 'OGA'],
      dtype='object', name='COD_TRG')

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb o

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  776
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Considered Spatial-Unit:  None
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER',
       'SAN', 'SAX', 'VMY', 'JEA', 'BON', 'CHA', 'VAI', 'VEN', 'MAC', 'GAR',
       'FOC', 'REP', 'GER', 'DEB', 'JAU', 'CPA', 'CRO', 'PAR', 'SOI', 'OGA'],
      dtype='object', name='COD_TRG')

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  481
Init U/Utarget size: torch.Size([2940, 40, 7])/t

ERROR 1: PROJ: proj_create_from_database: Open of /root/anaconda3/envs/pytorch-2.0.1/share/proj failed


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


AssertionError: Length of gdf = 0 while it should be = 1

## K-Fold Validation sur une config 'qui fait sens':

In [ ]:
from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation

trial_id = 'subway_in_config_good'
save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
epochs = 300
train_model_on_k_fold_validation(trial_id,load_config =False,save_folder=save_folder,epochs=epochs,hp_tuning_on_first_fold= True)

## K-Fold Validation sur une config d'HP tuning
- 500 samples
- 100 epochs max pour HP 
- grace period = 20 epochs 
- 100 epochs validations

In [4]:
from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation

trial_id  = 'subway_in_STGCN_MSELoss_2025_01_12_14_25_57312'
# 'subway_in_STGCN_MSELoss_2025_01_12_14_25_57312'
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_23_12_41192
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# 'subway_in_STGCN_MSELoss_2025_01_06_08_00_94523'
# 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963'

save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
epochs = 20 # 100
train_model_on_k_fold_validation(trial_id,load_config =True,save_folder=save_folder,epochs=epochs)


>>>> Load best CONFIG

Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683

 ===== ERROR ==== 
Try with torch >= 2.0.0 (works with 2.0.1) to allow 'prefetch_factor' 
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing

 ===== ERROR ==== 
Try with torch >= 2.0.0 (works with 2.0.1) to allow 'prefetch_factor' 
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing

 ===== ERROR ==== 
Try with torch >= 2.0.0 (works with 2.0.1) to allow 'prefetch_factor' 
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing

Init Dataset:  torch.Size([1710, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  68400 

U/Utarget size: torch

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Training Throughput:1943.49 sequences per seconds
>>> Training complete in: 0:00:10.760914
>>> Training performance time: min 0.013065814971923828 avg 0.014937877655029297 seconds (+/- 0.0020087536484890423)
>>> Loading performance time: min 0.00039076805114746094 avg 0.01231352210322738 seconds (+/- 0.03009291483050415)
>>> Forward performance time: 0.0038893495937520396 seconds (+/- 0.00028653499628730414)
>>> Backward performance time: 0.0058466784993902315 seconds (+/- 0.0010212611821145774)
>>> Plotting performance time: 6.123592979029606e-06 seconds (+/- 1.637296180504794e-05)
>>> Saving performance time: 0.17087674140930176 seconds (+/- 0.05150450382987147)
>>> PI-tracking performance time: 3.952729074578536e-06 seconds (+/- 1.1191267787282492e-06)
>>> Scheduler-update performance time: 4.496072468004728e-05 seconds (+/- 7.460435837465201e-05)
>>> Peak Power during training: 86.99 W)
>>> Validation time: 0:00:00.082697
Proportion of time consumed for Loading: 46.3%
Proportion o

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Training Throughput:2072.30 sequences per seconds
>>> Training complete in: 0:00:13.538053
>>> Training performance time: min 0.013666868209838867 avg 0.014702796936035156 seconds (+/- 0.0015655424369235332)
>>> Loading performance time: min 0.0005092620849609375 avg 0.010422396925181767 seconds (+/- 0.024234740329264638)
>>> Forward performance time: 0.0038660092577217854 seconds (+/- 0.00023806270691197932)
>>> Backward performance time: 0.005550487361162275 seconds (+/- 0.0008090821606841222)
>>> Plotting performance time: 5.345595510382402e-06 seconds (+/- 1.2915278337714558e-05)
>>> Saving performance time: 0.16926426887512208 seconds (+/- 0.053818907431212946)
>>> PI-tracking performance time: 4.203695999948602e-06 seconds (+/- 1.4225641271459064e-06)
>>> Scheduler-update performance time: 3.8887325086091695e-05 seconds (+/- 4.079396157184344e-05)
>>> Peak Power during training: 83.076 W)
>>> Validation time: 0:00:00.084059
Proportion of time consumed for Loading: 45.9%
Proporti

...


Training Throughput:1972.43 sequences per seconds
>>> Training complete in: 0:00:16.457918
>>> Training performance time: min 0.013089895248413086 avg 0.01486349105834961 seconds (+/- 0.0020489688267328436)
>>> Loading performance time: min 0.00037670135498046875 avg 0.00974876608247611 seconds (+/- 0.017363536678522116)
>>> Forward performance time: 0.003853910728063572 seconds (+/- 0.0002345102460702591)
>>> Backward performance time: 0.005696136704218686 seconds (+/- 0.0009234588766783176)
>>> Plotting performance time: 5.772239283511513e-06 seconds (+/- 1.3822776628719804e-05)
>>> Saving performance time: 0.1424289345741272 seconds (+/- 0.07701732165731649)
>>> PI-tracking performance time: 3.915084035773027e-06 seconds (+/- 6.440126228511379e-07)
>>> Scheduler-update performance time: 3.7469361957750823e-05 seconds (+/- 4.2948811790689096e-05)
>>> Peak Power during training: 83.55 W)
>>> Validation time: 0:00:00.089734
Proportion of time consumed for Loading: 48.5%
Proportion of 

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Training Throughput:1878.61 sequences per seconds
>>> Training complete in: 0:00:20.717155
>>> Training performance time: min 0.01367950439453125 avg 0.016904592514038086 seconds (+/- 0.0022291740493539216)
>>> Loading performance time: min 0.0005006790161132812 avg 0.009879193383245041 seconds (+/- 0.014181482910532832)
>>> Forward performance time: 0.0041238752205401685 seconds (+/- 0.00040509139333740545)
>>> Backward performance time: 0.006938916886506801 seconds (+/- 0.0017098214190255391)
>>> Plotting performance time: 7.1776540655838814e-06 seconds (+/- 1.772247683570146e-05)
>>> Saving performance time: 0.09758388996124268 seconds (+/- 0.08700883388519287)
>>> PI-tracking performance time: 5.0946285850123355e-06 seconds (+/- 1.604422740778131e-06)
>>> Scheduler-update performance time: 5.620404293662623e-05 seconds (+/- 9.407554532894058e-05)
>>> Peak Power during training: 83.536 W)
>>> Validation time: 0:00:00.101489
Proportion of time consumed for Loading: 46.6%
Proportion 

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Training Throughput:1898.71 sequences per seconds
>>> Training complete in: 0:00:26.281001
>>> Training performance time: min 0.012135982513427734 avg 0.015859603881835938 seconds (+/- 0.002311614395670667)
>>> Loading performance time: min 0.00037789344787597656 avg 0.00998743806202235 seconds (+/- 0.019723977256304096)
>>> Forward performance time: 0.003944347416831495 seconds (+/- 0.00040825133466293353)
>>> Backward performance time: 0.0063481743836788325 seconds (+/- 0.001543039528317876)
>>> Plotting performance time: 5.709497552168996e-06 seconds (+/- 1.3898300439431212e-05)
>>> Saving performance time: 0.17803452014923096 seconds (+/- 0.057473614035853327)
>>> PI-tracking performance time: 4.4421145790501645e-06 seconds (+/- 1.8808292567036314e-06)
>>> Scheduler-update performance time: 5.791061802914268e-05 seconds (+/- 0.00011914160701214796)
>>> Peak Power during training: 83.84 W)
>>> Validation time: 0:00:00.095595
Proportion of time consumed for Loading: 46.0%
Proportion

## Gestion des outliers: Estimation du Flux Max d'une station
- De Nombreux outliers présents à Perrache, d'autres assez élevée sur les autres stations. On va essayer de gérer ces outliers. 
    - On prend un Flux maximal enregistré : le Flux in max en 15min enregistré pendant la fête des lumière
        - Le Flux max est enregistré à Bellecour, autour de 2500 personnes / 15 minutes
        - Le second Flux max est enregistré à Charpennes, autour de 1600 personnes / 15 minutes
        - Le 3eme Flux max est enregsitré à Gorge de Loup, autour de 1600 personnes / 15 minutes
        - Aucunes des autres stations n'a dépassée 1400, même pendant la fête des lumières. 

- On choisit donc d'inputer la données, par interpolation linéaire, pour tout les flux dépassant:
    -  2500 pour bellecour
    - 1600 pour charpnnes et gorge de Loup
    - 1400 pour les autres stations.

In [ ]:
trial_id = 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# "subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963"
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# "subway_in_STGCN_MSELoss_2025_01_06_08_00_94523"
args,_ = load_configuration(trial_id,load_config=True,epochs=None)

# Comparaison des flux avec ce qu'on peut trouver sur les validations individuelles:

In [ ]:
import pandas as pd
import glob
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from constants.paths import FOLDER_PATH

list_files = glob.glob(f"{parent_dir}/{FOLDER_PATH}/validations_individuelles/*.txt")

# Load df :


path_txt = list_files[1]
df = pd.read_csv(path_txt, sep = "\t")
df['is_subway'] = df.EMP_CEB.transform(lambda x : x[:3])
df_subway = df[df['is_subway'] == 'VMM']   # Extract Funicular and Subway, starting both by "VMM"

df_subway['subway_mode'] = df_subway.EMP_CEB.transform(lambda x : x[:4])
df_subway = df_subway[df_subway['subway_mode'] != 'VMMF'] # Extract VVMx and remove VMMF (i.e. Subway Lane, excluding Funicular)
df_subway.VAL_DATE = pd.to_datetime(df_subway.VAL_DATE)

if False:
    # Load ref subway
    ref_subway = pd.read_csv(f"{folder_path}/ref_subway.csv").rename(columns = {'MEAN_X' : 'lon','MEAN_Y':'lat'})
    ref_subway = ref_subway[['lon','lat','COD_TRG','LIB_STA_SIFO']]

#Get Subway  
df_subway['subway_mode'] = df_subway.EMP_CEB.transform(lambda x : x[:4])
df_subway = df_subway[df_subway['subway_mode'] != 'VMMF'] # Extract VVMx (i.e. Subway Lane)
df_subway['COD_TRG'] = df_subway.EMP_CEB.transform(lambda x : x[5:8])
df_subway['lane'] = df_subway.EMP_CEB.transform(lambda x : x[3])

In [ ]:
df_subway_agg = df_subway.groupby([pd.Grouper(freq='15min',key='VAL_DATE'),'COD_TRG']).count().iloc[:,0]
df_subway_agg.name = 'Flow'
df_subway_agg = df_subway_agg.reset_index()

#df_subway_agg[~df_subway_agg['COD_TRG'].isin(['CHA','BEL','GOR'])].pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()
df_subway_agg[df_subway_agg['COD_TRG'].isin(['HOT','BEL','CHA','AMP','PER','PAR'])].pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()
#df_subway_agg.pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()

## Génération de la configuration, choix des modification, entrainement et visalisation.
Args: 
--
- station : définie les station qui vont être regardée (visualisation profil time série réelle et prédiction)
- set_spatial_units: défini les stations qui vont être prise en compte lors de l'entrainement. La taille de 'set_spatial_unit' est donc la taille du graphe. Si non précisé, on fait la prédiction sur les 40 STATION du réseau.

In [4]:
args.hp_tuning_on_first_fold

True

## Best Config Subway-IN  sur 500 Sample 

In [ ]:
# Init:
dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100,
  'lr':1.45e-3,
  'weight_decay':0.060384,
  'dropout': 0.368853,
  'temporal_h_dim': 16,
  'spatial_h_dim': 256,
  'output_h_dim': 256,	
  'scheduler': None,
    'vision_model_name': 'VariableSelectionNetwork',
    'vision_concatenation_early':True,   
    'vision_concatenation_late':True,
    'vision_num_heads':3,
    'vision_grn_out_dim':12,
 }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

## Best Config Subway-IN / Subway-OUT sur 500 Sample 
Uitlisation de cette config sur 'netmob_POIs'. Même ordre de grandeur de paramètres, on pourrait avoir le même type d'HP tuning (réponse: Non)

In [ ]:
# Init:
dataset_names = ["subway_in","netmob_POIs"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100,
  'lr':1.45e-3,
  'weight_decay':0.060384,
  'dropout': 0.368853,
  'temporal_h_dim': 16,
  'spatial_h_dim': 256,
  'output_h_dim': 256,	
  'scheduler': None,
    'vision_model_name': 'VariableSelectionNetwork',
    'vision_concatenation_early':True,   
    'vision_concatenation_late':True,
    'vision_num_heads':3,
    'vision_grn_out_dim':12,
 }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

## Best Config Subway-IN / Subway-OUT sur 500 Sample 

In [ ]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

# Init:
dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100, #100
                'lr':4e-4,# 4e-4,
                'weight_decay':0.05,
                'dropout':0.1,
                #'set_spatial_units':  station,  
                #   
                #'scheduler':None,
                'scheduler':True,
                'torch_scheduler_milestone': 2,
                'torch_scheduler_gamma':0.99,
                'torch_scheduler_lr_start_factor':0.1,

                'vision_num_heads':4,
                "vision_grn_out_dim":32,
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':True,   
                'vision_concatenation_late':True,
                           }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

## Pattern d'erreur selon les stations : 

In [ ]:
# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(ds.spatial_unit)),
                                                limit_percentage_error = limit_percentage_error)

In [ ]:
trainer.args.epochs = 20
trainer.train_and_valid(mod = 1000,mod_plot = None) 
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

import pandas as pd 
pd.DataFrame(dict(valid = trainer.valid_loss)).iloc[-120:].plot()

In [ ]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['BEL','PAR','AMP','FLA']   # 'BON'  #'GER'
# ...

modification = {'epochs' : 100, #100
                'lr':4e-4,
                #'set_spatial_units': station,#['CHA','GER','BON','SOI'],
                #'TE_concatenation_late':False,
                #'TE_concatenation_early':True,   
                #'TE_embedding_dim' : 16, # 3
                #'TE_multi_embedding' : True , 
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':False,   
                'vision_concatenation_late':True,
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(
                                                   station = station,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

# Init
if False:
    for training_mode in training_mode_to_visualise:
        min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
        limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
        fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                    freq='1h',
                                                    min_flow=min_flow,
                                                    figsize = (30,5*len(ds.spatial_unit)),
                                                    limit_percentage_error = limit_percentage_error)

In [ ]:
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

In [ ]:
trainer.train_and_valid(mod = 1000,mod_plot = None) 
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

In [ ]:
from plotting.plotting import plot_attn_weight
from constants.paths import CALENDAR_TYPE
nb_calendar_data = len(CALENDAR_TYPE)

training_mode = 'test'
temporal_agg = 'weekday_hour' # 'hour' # 'weekday' # 'weekday_hour'
save = 'attn_weight'
plot_attn_weight(trainer,nb_calendar_data,ds,training_mode,temporal_agg,save)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Supposons que attn_weights est votre tenseur de poids d'attention
# attn_weights : Shape [B, 1, P]
# Prenons le premier exemple du batch
attn_weights_example = attn_weights[0].squeeze(0)  # Shape [P]

# Normaliser les poids d'attention pour la visualisation
attn_weights_example = attn_weights_example.detach().cpu().numpy()  # Convertir en numpy

# Créer une figure
plt.figure(figsize=(10, 6))

# Afficher les poids d'attention
plt.bar(range(len(attn_weights_example)), attn_weights_example, color='skyblue')
plt.title('Poids d\'attention pour l\'exemple 1')
plt.xlabel('Séries temporelles')
plt.ylabel('Poids d\'attention')

# Ajouter des ticks pour les séries temporelles
plt.xticks(range(len(attn_weights_example)), [f'Série {i}' for i in range(len(attn_weights_example))], rotation=45)

# Afficher la figure
plt.tight_layout()
plt.show()

In [ ]:
from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['PAR']   # 'BON'  #'GER'

modification = {'epochs' : 250, #100
                'lr':1e-4,
                #'set_spatial_unit': station
                }

training_mode_to_visualise = ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(
                                                  station = station,
                                                  modification=modification,
                                                  training_mode_to_visualise=training_mode_to_visualise)

In [ ]:
# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(ds.spatial_unit)),
                                                limit_percentage_error = limit_percentage_error)

In [ ]:
import torch
import pandas as pd
index_perrache = list(ds.spatial_unit).index('PER')
inputs = [[x,y,x_c] for  x,y,x_c in ds.dataloader['test']]
X = torch.cat([x for x,_,_ in inputs],0)
Y = torch.cat([y for _,y,_ in inputs],0)
#X_c = torch.cat([x_c for x,_,_ in inputs],-1)
l = 96
pd.DataFrame({'PER_TRUE': Y[:,index_perrache,0].detach().clone().reshape(-1),
              #'t-1': X[:,index_perrache,-1].detach().clone().reshape(-1),
              #'t-2': X[:,index_perrache,-2].detach().clone().reshape(-1),
              #'t-3': X[:,index_perrache,-3].detach().clone().reshape(-1),
              #'t-4': X[:,index_perrache,-4].detach().clone().reshape(-1),
              #'t-5': X[:,index_perrache,-5].detach().clone().reshape(-1),
              #'t-6': X[:,index_perrache,-6].detach().clone().reshape(-1),
              't-d': X[:,index_perrache,-7].detach().clone().reshape(-1),
              }).iloc[l:l+7*l].plot()


In [ ]:
from examples.train_and_visu_non_recurrent import evaluate_config
dataset_names = ["subway_in","calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'
modification = {'epochs' : 400, #100
                'lr':1e-4,
                'set_spatial_units': ['CHA','GER','BON','SOI'],#['CHA','GER','BON','SOI'],
                'TE_concatenation_late':False,
                'TE_concatenation_early':True,   
                'TE_embedding_dim' : 2,
                'TE_multi_embedding' : True ,         
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

## Evaluate on non recurrent event: 
### Visualisation: 
#### Cas `subway_in` stade du Lou Gerland avec un match de Rugby : 

## HP tuning: 

In [ ]:
from examples.Total_evaluation_of_model import HP_and_valid_one_config,hyperparameter_tuning
from examples.benchmark import local_get_args
import pickle

model_name = 'STGCN' #'CNN'
dataset_for_coverage = ['subway_in','netmob_POIs'] 
dataset_names = ['calendar']
vision_model_name = None

args,_,_ = local_get_args(model_name,
                        args_init = None,
                        dataset_names=dataset_names,
                        dataset_for_coverage=dataset_for_coverage,
                        modification = {'ray':True,
                                        'grace_period':2,
                                        'HP_max_epochs':10,
                                        'evaluate_complete_ds' : True,
                                        'set_spatial_units' : ['BON','SOI','GER','CHA'],
                                        'vision_model_name': None
                                        })

# Init 
epochs_validation = 30
num_samples = 10

# HP and evaluate K-fold best config
#HP_and_valid_one_config(args,epochs_validation,num_samples)
analysis,trial_id = hyperparameter_tuning(args,num_samples)
print('trial_id: ',trial_id)



## Load HP tuning, have a look on the best configurations:

In [1]:
import pickle 
import pandas as pd 

# 2000 Trial et 100 Epochs: 
trial_ids = ['subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963','subway_in_STGCN_MSELoss_2025_01_06_08_00_94523']

# 'subway_in_STGCN_MSELoss_2024_12_11_00_16_42861'   -> 'Seulement' 400 trials, et 50 epochs 
for trial_id in trial_ids:
    columns_to_drop = ['time_this_iter_s', 'done', 'date', 
                    'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip',
                    'time_since_restore', 'iterations_since_restore', 'experiment_tag']
    path_csv = f'save/HyperparameterTuning/{trial_id}.csv'
    path_pickle = 'save/HyperparameterTuning/model_args.pkl'

    df_hp_tuning = pd.read_csv(path_csv)
    model_args = pickle.load(open(path_pickle,'rb'))


    metric = 'Loss_model'

    dict_rename = {key: key.split('/')[-1] for key in df_hp_tuning.columns}
    df_hp_tuning = df_hp_tuning.rename(columns = dict_rename)
    df_hp_tuning = df_hp_tuning.drop(columns = columns_to_drop)


    k=10
    print(df_hp_tuning.shape)
    display(df_hp_tuning.sort_values(metric).iloc[:k])



(500, 17)


,trial_id,training_iteration,Loss_model,lr,weight_decay,dropout,temporal_h_dim,spatial_h_dim,output_h_dim,scheduler,concatenation_early,concatenation_late,num_heads,grn_out_dim,torch_scheduler_milestone,torch_scheduler_gamma,torch_scheduler_lr_start_factor
160,87851_00160,100,0.005920,0.00145,0.060384,0.368853,16,256,256,NaN,True,True,3,12,NaN,NaN,NaN
451,87851_00451,100,0.006093,0.00125,0.009742,0.491189,32,128,64,True,False,True,1,4,5.0,0.992034,0.828268
69,87851_00069,100,0.006127,0.00455,0.014063,0.106061,8,32,8,True,False,True,2,8,27.0,0.990475,0.145083
259,87851_00259,100,0.006154,0.00060,0.073119,0.390112,16,64,32,True,False,True,6,24,18.0,0.987133,0.781067
20,87851_00020,100,0.006339,0.00225,0.034687,0.529574,32,32,16,True,False,True,1,16,17.0,0.996583,0.589259
53,87851_00053,100,0.006392,0.00035,0.068259,0.275934,16,256,16,NaN,True,False,1,16,NaN,NaN,NaN
176,87851_00176,100,0.006405,0.00025,0.034697,0.060886,256,64,32,True,False,True,6,24,25.0,0.997865,0.714208
359,87851_00359,100,0.006409,0.00175,0.014504,0.131933,256,256,64,True,True,False,6,192,17.0,0.987810,0.487705
121,87851_00121,100,0.006574,0.00120,0.025116,0.568096,32,128,64,NaN,True,False,2,8,NaN,NaN,NaN
189,87851_00189,100,0.006692,0.00365,0.015256,0.285538,32,8,256,True,True,False,2,64,3.0,0.997815,0.115074


(500, 13)


,trial_id,training_iteration,Loss_model,lr,weight_decay,dropout,temporal_h_dim,spatial_h_dim,output_h_dim,scheduler,torch_scheduler_milestone,torch_scheduler_gamma,torch_scheduler_lr_start_factor
74,d02e7_00074,100,0.005833,0.00110,0.029826,0.303705,16,8,16,NaN,NaN,NaN,NaN
425,d02e7_00425,100,0.005834,0.00070,0.032901,0.116875,8,128,16,True,19.0,0.995579,0.657189
430,d02e7_00430,100,0.005911,0.00445,0.053486,0.550622,64,8,128,NaN,NaN,NaN,NaN
497,d02e7_00497,100,0.005931,0.00250,0.094097,0.430751,32,64,16,NaN,NaN,NaN,NaN
3,d02e7_00003,100,0.005935,0.00055,0.090834,0.148635,32,32,256,NaN,NaN,NaN,NaN
13,d02e7_00013,100,0.006026,0.00205,0.068388,0.112663,8,128,8,NaN,NaN,NaN,NaN
434,d02e7_00434,100,0.006069,0.00090,0.057198,0.481282,32,256,8,NaN,NaN,NaN,NaN
38,d02e7_00038,100,0.006073,0.00240,0.050927,0.525008,16,8,256,NaN,NaN,NaN,NaN
451,d02e7_00451,100,0.006330,0.00055,0.029741,0.225671,16,128,256,True,23.0,0.997041,0.786742
384,d02e7_00384,100,0.006378,0.00100,0.047268,0.451278,128,16,256,NaN,NaN,NaN,NaN


In [ ]:
save_folder = 'save/K_fold_validation/training_with_HP_tuning'

for trial_id in trial_ids:
    print(f'\n{trial_id}')
    metric_csv = f'{save_folder}/METRICS_{trial_id}.csv'
    df_metrics_k_fold_valid = pd.read_csv(metric_csv,index_col =0)

    losses_csv = f'{save_folder}/Losses_{trial_id}.csv'
    df_losses_k_fold_valid = pd.read_csv(losses_csv,index_col =0)

    display(df_metrics_k_fold_valid)
    usefull_columns = [c for c in df_losses_k_fold_valid.columns if 'valid' in c]
    df_losses_k_fold_valid[usefull_columns].iloc[10:].plot()

In [ ]:
import numpy as np 
from examples.train_model_on_k_fold_validation import train_valid_K_models,get_model_metrics
from examples.benchmark import local_get_args


# TE modification: 
TE_modification = {'TE_concatenation_early':True,
                  'TE_multi_embedding':True,
                  'TE_concatenation_late': False,
                  'TE_out_h_dim':4,
                  'TE_embedding_dim':4
               }


for dataset_names,te_modif in zip([['subway_in','calendar'],['subway_in']],[TE_modification,{}]):
   ### === Modification :
   modification = {'evaluate_complete_ds':True,
                  'temporal_h_dim':32,
                  'spatial_h_dim':8,
                  'output_h_dim':64,
                  'dropout':5e-2,
                  'weight_decay':5e-3,
                  'lr':5e-4,
                  'epochs':2
                  }
   modification.update(te_modif)
   ### _____________________

   # Load Config
   args,_,_ = local_get_args(model_name= 'STGCN',
                           args_init=None,
                           dataset_names= dataset_names,
                           dataset_for_coverage= ['subway_in','netmob_POIs'] ,
                           modification=modification
                           )

   # Train, Valid: K-fold and complete ds 
   trainer,args,valid_losses,training_mode_list,metric_list,df_loss = train_valid_K_models(args,
                                                                                          folds=np.arange(args.K_fold),
                                                                                          hp_tuning_on_first_fold=True,
                                                                                          trial_id= None,
                                                                                          save_folder=None)
   # Get metrics: 
   df_results,df_metrics =  get_model_metrics(trainer,args,valid_losses,training_mode_list,metric_list)

   display(df_results)
   display(df_metrics)

   df_losses = pd.DataFrame({'Valid_loss':trainer.valid_loss,
               'Train_loss':trainer.train_loss
               })[10:]

   df_losses.plot()

In [ ]:
from examples.train_model_on_k_fold_validation import load_configuration
from examples.train_and_visu_non_recurrent import evaluate_config

trial_id = 'calendar_STGCN_MSELoss_2024_12_17_21_18_13719'
best_args,folds = load_configuration(trial_id,load_config=True,epochs=None)

dict_keys_embedding = {'TE_concatenation_order/concatenation_late':'TE_concatenation_late',
            'TE_concatenation_order/concatenation_early':'TE_concatenation_early',
             'TE_fc1/fc1':'TE_fc1', 
             'TE_fc1/fc2':'TE_fc2', 
             'TE_fc1/activation_fc1':'TE_activation_fc1',
             }

for key in dict_keys_embedding.keys():
    value = vars(best_args)[key]
    new_key = dict_keys_embedding[key]
    setattr(best_args,new_key,value)


modification = vars(best_args)


dataset_names = ["calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'

(trainer,ds,ds_no_shuffle,args) = evaluate_config(transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification,station=station)

In [ ]:
dataset_names = ["calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  None

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'
modification = {'epochs' : 20, #100
                'lr':1e-4,
                'temporal_h_dim' : 32,
                'spatial_h_dim' : 16,
                'output_h_dim' : 32,
                'TE_embedding_dim': 16,
                'set_spatial_units': ['CHA','GER','BON','SOI'],
                'TE_out_h_dim': 16,
                'TE_concatenation_late':False,
                'TE_concatenation_early':True,
                           }
(trainer,ds,ds_no_shuffle,args) = evaluate_config(vision_model_name,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification)


# lr 5e-3
# 32 / 32 / 32 - 3.5e-2   , 380 000
# 32 / 16 / 32 - 3.5e-2   , 380 000

#### Cas `subway_out` stade du Lou Gerland avec un match de Rugby : 

#### Cas `subway_in` Laurent Bonnevay Astroballe avec un match de Basket : 

#### Cas `subway_out` Laurent Bonnevay Astroballe avec un match de Basket : 

## Maintenant on va intégrer les données NetMob et voir si elles permettent de réduire l'erreur de prédiction :

In [1]:
import os 
import pickle 
import pandas as pd 

trial_id = 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2024_12_07_02_55_5679'
#'subway_in_calendar_STGCN_MSELoss_2024_12_12_15_51_46099'
#'subway_in_calendar_STGCN_MSELoss_2024_12_12_14_16_71587'

path_csv = f'save/HyperparameterTuning/{trial_id}.csv'
path_pickle = 'save/HyperparameterTuning/model_args.pkl'
df_hp_tuning = pd.read_csv(path_csv)
model_args = pickle.load(open(path_pickle,'rb'))

metric = '_metric/Loss_model'

best_model = df_hp_tuning.sort_values(metric).iloc[0]
HP_args = [indx.replace('config/', '') for indx in best_model.index if 'config/' in indx]
args = model_args['model'][trial_id]['args']

In [ ]:
print(df_hp_tuning.columns)

df_hp_tuning.sort_values(metric)[['_metric/Loss_model', 'config/lr', 'config/weight_decay',
       'config/dropout', 'config/vision_vision_grn_out_dim']].iloc[:30]

In [ ]:
dataset_names = ["subway_in","netmob_image_per_station"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  'ImageAvgPooling'

(trainer,ds,ds_no_shuffle,args) = evaluate_config(vision_model_name)

#### Prediction avec uniquement NetMob, sans subway-in: 

#### Prediction avec uniquement Calendar, sans subway-in: 